In [3]:
import os
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('solution_ex_10_v12').getOrCreate()

22/07/18 15:12:52 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/18 15:12:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/18 15:12:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/18 15:12:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/18 15:12:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
inputPath = './data'
outputPath = './output_ex_10_v2'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [6]:
fileRDD = spark.sparkContext.textFile(inputPath)

In [7]:
# Apply a filter transformation to select only the lines with PM10>50
readingsHighValueRDD = fileRDD.filter(lambda PM10: float(PM10.split(',')[2]) > 50 )

In [8]:
# Create an RDD of key-value pairs
sensorsCriticalDatesRDD = readingsHighValueRDD.map(lambda PM10Reading: (PM10Reading.split(',')[0], PM10Reading.split(',')[1]) )

In [9]:
finalSensorCriticalDates = sensorsCriticalDatesRDD.groupByKey()

In [10]:
# The map method is used to transform the content of the iterable over the values of each key into a list (that can be stored in a readable format)
criticalDateStringFormat = finalSensorCriticalDates.mapValues(lambda dates : list(dates))

In [11]:
# get all sensors ID from the complete input file
allSensorsRDD = fileRDD.map(lambda PM10Reading: PM10Reading.split(',')[0])

In [12]:
# get the identifiers of the sensors that have never been associated with a PM10 values greater than 50
sensorsNeverHighValueRDD = allSensorsRDD.subtract(finalSensorCriticalDates.keys())

In [13]:
# create tuples of sensorID neve associated with hiv PM10 values
sensorsNeverHighValueRDDEmptyList = sensorsNeverHighValueRDD.map(lambda sensorId: (sensorId, []))

In [14]:
# Compute the final result
resultRDD = criticalDateStringFormat.union(sensorsNeverHighValueRDDEmptyList)

In [15]:

resultRDD.saveAsTextFile(outputPath)